In [1]:
import numpy as np
import pandas as pd
import os
from joblib import load

# 백테스트 데이터 로드
backtesting_path = 'backtesting_raw.xlsx'
backtesting_data = pd.read_excel(backtesting_path, sheet_name=None)

# 인덱스를 날짜 형식으로 변환 후 문자열로 변경
for keyword, df in backtesting_data.items():
    # 원본 인덱스 제거
    df.reset_index(drop=True, inplace=True)
    # 인덱스를 날짜 형식으로 변환
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    # 인덱스를 'date' 열로 설정
    df.set_index('date', inplace=True)
    print(f"{keyword} 키워드의 백테스트 데이터:")
    print(df.head())


auto 키워드의 백테스트 데이터:
              WTI  DGS2  DGS10  TIPS    VIX    PPI    per    pbr     원달러  \
date                                                                       
2010-05-03  86.19  1.00   3.72  1.32  20.19  173.9  15.23  15.23  1108.4   
2010-06-01  72.70  0.78   3.29  1.32  20.19  173.9  15.23  15.23  1108.4   
2010-07-01  72.95  0.63   2.96  1.21  28.53  176.1  15.45  15.45  1227.7   
2010-08-02  81.25  0.56   2.99  1.29  33.70  175.2  14.86  14.86  1153.8   
2010-09-01  73.97  0.50   2.86  1.04  21.74  174.8  14.96  14.96  1235.9   

                 원엔  ...   구리선물_60  중국고정자산투자(yoy%)_60  중국 PMI_60   금 선물_60  \
date                 ...                                                     
2010-05-03  1186.77  ...  0.002464          -0.003053  -0.000077  0.001188   
2010-06-01  1186.77  ...  0.002464          -0.003053  -0.000077  0.001188   
2010-07-01  1337.19  ...  0.002464          -0.003053  -0.000077  0.001188   
2010-08-02  1227.29  ...  0.002464          -0.003053  -0

In [2]:
# 저장된 모델 로드
model_dir = "plk_folder_05_new"
loaded_models = {}
for keyword in backtesting_data.keys():
    model_path = os.path.join(model_dir, f"{keyword}_model.pkl")
    if os.path.exists(model_path):
        loaded_models[keyword] = load(model_path)
        print(f"{keyword} 키워드의 모델을 {model_path}에서 로드했습니다.")
    else:
        print(f"{keyword} 키워드의 모델 파일이 {model_path}에 존재하지 않습니다.")

auto 키워드의 모델을 plk_folder_05_new\auto_model.pkl에서 로드했습니다.
construct 키워드의 모델을 plk_folder_05_new\construct_model.pkl에서 로드했습니다.
capital_market 키워드의 모델을 plk_folder_05_new\capital_market_model.pkl에서 로드했습니다.
chemicals 키워드의 모델을 plk_folder_05_new\chemicals_model.pkl에서 로드했습니다.
equipment 키워드의 모델을 plk_folder_05_new\equipment_model.pkl에서 로드했습니다.
transport 키워드의 모델을 plk_folder_05_new\transport_model.pkl에서 로드했습니다.
semi 키워드의 모델을 plk_folder_05_new\semi_model.pkl에서 로드했습니다.
bank 키워드의 모델을 plk_folder_05_new\bank_model.pkl에서 로드했습니다.
steel 키워드의 모델을 plk_folder_05_new\steel_model.pkl에서 로드했습니다.
telecom 키워드의 모델을 plk_folder_05_new\telecom_model.pkl에서 로드했습니다.
staples 키워드의 모델을 plk_folder_05_new\staples_model.pkl에서 로드했습니다.
discretionary 키워드의 모델을 plk_folder_05_new\discretionary_model.pkl에서 로드했습니다.
kospi 키워드의 모델을 plk_folder_05_new\kospi_model.pkl에서 로드했습니다.


In [3]:
# 각 모델의 예측이 특정 패턴을 따르는 경우, 예측 결과가 동일하게 나타날 수 있습니다.
#예를 들어, 모델이 특정한 임계값을 기준으로 예측을 수행하는 경우, 입력 데이터가 그 임계값을 넘지 않는다면 결과는 동일할 수 있습니다.
# 즉 우리가 수행한 모델이 일정 값을 넘기면 S_buy buy sell이므로 오류가 생겼던 것 같ㅇ므
from sklearn.preprocessing import StandardScaler

# 각 행에 대해 예측 수행
all_predictions = {}

for keyword, df in backtesting_data.items():
    if keyword in loaded_models:
        model = loaded_models[keyword]
        predictions = []
        scaler = StandardScaler()
        df_scaled = scaler.fit_transform(df)
        for idx, row in enumerate(df_scaled):
            date = df.index[idx]  # 인덱스가 날짜 형식임
            X_test = row.reshape(1, -1)
            probabilities = model.predict_proba(X_test)[0]
            predictions.append({
                'date': date,
                'S_buy': probabilities[0],
                'buy': probabilities[1],
                'sell': probabilities[2]
            })
        all_predictions[keyword] = pd.DataFrame(predictions)
        print(f"\n{keyword} 키워드의 예측 결과:")
        print(all_predictions[keyword].head())


auto 키워드의 예측 결과:
         date     S_buy       buy      sell
0  2010-05-03  0.352797  0.431460  0.215743
1  2010-06-01  0.283069  0.429759  0.287172
2  2010-07-01  0.283069  0.501188  0.215743
3  2010-08-02  0.283069  0.572616  0.144315
4  2010-09-01  0.353545  0.502140  0.144315

construct 키워드의 예측 결과:
         date     S_buy       buy      sell
0  2010-05-03  0.106656  0.163389  0.729955
1  2010-06-01  0.106656  0.163389  0.729955
2  2010-07-01  0.212201  0.369142  0.418657
3  2010-08-02  0.160085  0.265943  0.573971
4  2010-09-01  0.106656  0.163389  0.729955

capital_market 키워드의 예측 결과:
         date     S_buy       buy      sell
0  2010-05-03  0.182365  0.273199  0.544437
1  2010-06-01  0.182365  0.273199  0.544437
2  2010-07-01  0.207764  0.169371  0.622865
3  2010-08-02  0.201508  0.246097  0.552394
4  2010-09-01  0.209210  0.335421  0.455368

chemicals 키워드의 예측 결과:
         date     S_buy       buy      sell
0  2010-05-03  0.079865  0.343334  0.576800
1  2010-06-01  0.133509  0.3

In [4]:
# 예측 결과를 엑셀 파일로 저장
output_path = "predictions_v3.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for keyword, predictions_df in all_predictions.items():
        predictions_df.to_excel(writer, sheet_name=keyword, index=False)
print(f"예측 결과가 {output_path}에 저장되었습니다.")

예측 결과가 predictions_v3.xlsx에 저장되었습니다.
